In [1]:

import numpy as np
import subprocess
import os
import sys
import shutil, fnmatch
from linecache import getline, clearcache
from jupyprint import jupyprint  
from IPython.display import display, FileLink, FileLinks, HTML
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
from clawpack.visclaw import animation_tools

In [2]:
from os.path import expanduser
home = expanduser("~")

#répertoire_travail  = os.path.join(HOME, 'run_lac_6/')
répertoire_travail  = os.getcwd()
répertoire_résultat = os.path.join(répertoire_travail, '_output')
sys.path.insert(0,répertoire_travail)  

print('Répertoire où sont les données : \n  %s' % répertoire_résultat)
if not os.path.isdir(répertoire_travail):
      print('*** Revoir le nom du répertoire (je ne le trouve pas)')

fichiers = fnmatch.filter(os.listdir(répertoire_résultat), 'fort.q*' )
fichiers.sort()
NbSim    = len(fichiers)
print(f"Il y a {NbSim} fichiers fort.q*.")

Répertoire où sont les données : 
  /home/ancey/6_Burgers_classic_src/_output
Il y a 51 fichiers fort.q*.


In [3]:

clearcache()
source = répertoire_résultat+'/'+fichiers[0]
header = [getline(source, i) for i in range(1, 6)]
values = [float(h.split()[0].strip()) for h in header]
keys = [ (h.split()[-1].strip()) for h in header]
dictionnaire = {keys[k]:values[k] for k in range(0,5)}
xlow = dictionnaire['xlow']
mx = int(dictionnaire['mx'])
dx = dictionnaire['dx']
nbsim = len(fichiers)
xupper = xlow+mx*dx

fichiers_t = fnmatch.filter(os.listdir(répertoire_résultat), 'fort.t*' )
fichiers_t.sort()
tmax = float((getline(répertoire_résultat+'/'+fichiers_t[-1],1).split())[0])
dt = tmax/(NbSim-1)
jupyprint(" $x_{min}$ ="+f" {xlow}")
jupyprint(" $x_{max}$ ="+f" {xupper}")
jupyprint(" $\\mathrm{d}x $ ="+f" {dx}")
jupyprint(f"mx = {mx}")
jupyprint(" $t_{max}$ ="+f" {tmax:.2f}")
jupyprint(" $\\mathrm{d}t $ ="+f" {dt:.2f}")

source = répertoire_travail+'/setprob.data'
header = [getline(source, i) for i in range(7, 10)]
values = [float(h.split()[0].strip()) for h in header]
keys = [ (h.split()[-1].strip()) for h in header]
dictionnaire = {keys[k]:values[k] for k in range(0,3)}
mu = dictionnaire['mu']
xlim = int(dictionnaire['xlim'])
a = dictionnaire['a']
jupyprint(" $\\mu $ ="+f" {mu}")
jupyprint(" $x_{lim} $ ="+f" {xlim}")
jupyprint(" $a $ ="+f" {a}")

 $x_{min}$ = 0.0

 $x_{max}$ = 2.0

 $\mathrm{d}x $ = 0.002

mx = 1000

 $t_{max}$ = 1.00

 $\mathrm{d}t $ = 0.02

 $\mu $ = 5.0

 $x_{lim} $ = 1

 $a $ = 1.0

In [4]:
frames = np.zeros((NbSim,mx))
for i in range(0,NbSim):
    f = fichiers[i]
    data = np.loadtxt(répertoire_résultat+'/'+f,skiprows=6)
    frames[i] = data

In [5]:
x = np.linspace(xlow,xupper,mx)
figs = []
hauteurs = []
hmax = frames.max()
L  = (xupper+xlow)/2
temps_in = np.linspace(0,tmax,NbSim)

# Solution théorique
def xf(t,a,xlim,mu):
    if mu>0:
        xfr = (xlim*np.sqrt(-a*np.exp(-t*mu ) +  a + mu ))/np.sqrt(mu)
    else:
        xfr = xlim*np.sqrt(1+a*t)
    return xfr
#((np.exp((t*mu)))**-0.5)*((mu**-0.5)*(np.sqrt((((np.exp((t*mu)))*(a+mu))-a))))*xlim
#

def sol(x,t,a,xlim,mu):
    s = 0
    
    if (x<=xf(t,a,xlim,mu)):
            if mu > 0:
                s = (a*((np.exp((-mu*t)))*x))/((1.+(a/mu))-((a*(np.exp((-mu*t))))/mu))  #a*x*np.exp(-mu*t)/(1+a*(1-np.exp(-mu*t)))
            else:
                 s = a*x/(1+a*t)
    return s



def ExportAnimation():
    
   
    for i in range(0,NbSim):
         
        h_i = frames[i]
        
        hauteurs.append(h_i)
        fig  = plt.figure( figsize=(10,2))
        axes = plt.subplot(1, 1, 1)
        
      
        axes.set_ylim((0,hmax))

        
        axes.set_xlabel(r'$x $',fontsize=14)
        axes.set_ylabel(r'$u(x,t) $',fontsize=14)
         

        text = axes.text(L/2,  0.8*hmax, '')
        tt = temps_in[i]
        sol_i = [sol(xi,tt,a,xlim,mu) for xi in x]
        val = f'{tt:.1f}'
        text.set_text(r'$ t = {} $ s '.format(val))
         
         

        axes.set_title(" ")
      

        axes.plot(x , h_i, color = 'deepskyblue',label='numerical solution')
        axes.plot(x , sol_i, color = 'red', linestyle = '--',label='theoretical solution')
        #axes.plot(distance, eta_i, 'b')
        figs.append(fig)
        plt.close(fig)
        axes.legend()
    return figs

In [6]:
figures = ExportAnimation()
animation_tools.interact_animate_figs(figures, manual=True)

interactive(children=(IntSlider(value=0, description='frameno', max=50), Button(description='Run Interact', st…

In [7]:
# Animation
images = animation_tools.make_images(figures)
anim = animation_tools.animate_images(images, figsize=(12,4))
HTML(anim.to_jshtml())

In [56]:
figures[5].savefig('burgers-linear-src.png',dpi=300,bbox_inches='tight')